# Compute the PRR for a given pair drug-adverse reaction

In [1]:
require 'pg'

true

In [2]:
conn = PG::Connection.open(host: "postgres", 
                      port: 5432, 
                      user: "postgres",
                      password: "mysecretpassword",
                      dbname: "adr")

#<PG::Connection:0x000056533b909f68>

Retrieve All the Drug/Reaction pairs with count

In [4]:
query = 
"SELECT D.name AS drug, A.name_en AS reaction, count(*) AS n
FROM 
mpd.pharmaceutical_products AS D
JOIN 
mpd.medicinal_products AS BD
ON 
D.id = BD.pharmaceutical_product_id 
JOIN
reports.treatments  AS T
ON 
BD.id = T.mpd_medicinal_product_id
JOIN 
reports.reports  AS R
ON 
T.report_id = R.id
JOIN 
reports.meddra_adrs AS M
ON 
R.id = M.report_id
JOIN 
meddra.low_level_terms AS A
ON
A.id  = M.meddra_low_level_term_id
GROUP BY D.name, A.name_en
ORDER BY count(*) DESC"

nil

Exercise: add ATC code to the results above si the structure become (drug, reaction, ATC, n)

In [ ]:
r = conn.exec(query)

In [14]:
adrs = []; r.each{|row| row["n"]= row["n"].to_i; adrs.push(row) } 

#<PG::Result:0x000056533be64e40 status=PGRES_TUPLES_OK ntuples=311964 nfields=3 cmd_tuples=311964>

In [15]:
adrs[0]

{"drug"=>"WARFARIN SODICO", "reaction"=>"Epistaxis", "n"=>2036}

In [16]:
drug, reaction, a = adrs[0]["drug"], adrs[0]["reaction"], adrs[0]["n"]

["WARFARIN SODICO", "Epistaxis", 2036]

In [20]:
c = adrs.select do |tuple| 
                      tuple["drug"] == drug && 
                      tuple["reaction"] != reaction 
                end.map{|tuple| tuple["n"] }.reduce(:+)

18959

Exercise: compute b and d in an anlogous way